In [1]:
from simulator.bb84 import simulation_bb84
from tqdm import tqdm
from numpy import arange

In [9]:
results_list = []
for qber in tqdm(arange(start=0.01, stop=0.16, step = 0.01)):
    simulation_results = simulation_bb84(
        alice_basis_length=500,
        disturbance_probability=qber,
        cascade_n_passes=4
    )
    results_list.append(simulation_results)
    # How long will it take to perform this simple test?

 20%|██        | 3/15 [00:00<00:01,  7.07it/s]

KeyError for binary_previous
{'5': '0', '156': '0', '162': '0', '187': '1', '79': '1', '64': '0', '41': '1', '160': '0', '13': '1', '26': '1', '9': '0', '78': '1', '27': '1', '18': '1', '91': '0', '111': '1', '96': '0', '55': '0', '25': '1', '22': '1', '151': '1', '102': '0', '153': '0', '98': '1', '45': '1', '47': '0', '61': '0', '48': '1', '161': '0', '169': '0', '117': '1', '62': '1', '106': '0', '145': '1', '183': '1', '49': '1', '100': '1', '191': '0', '103': '0', '86': '0', '1': '1', '173': '1', '144': '1', '37': '0', '189': '1', '6': '1', '193': '1', '17': '1', '36': '1', '89': '0', '136': '1', '97': '0', '122': '1', '87': '1', '192': '0', '194': '1', '50': '0', '131': '1', '149': '1', '115': '1', '128': '1', '159': '1', '124': '1', '67': '1', '77': '1', '133': '1', '85': '1', '175': '0', '65': '1', '99': '1', '21': '0', '177': '0', '69': '1', '33': '1', '71': '0', '63': '0', '172': '0', '32': '0', '119': '0', '23': '1', '158': '0', '147': '1', '121': '0', '51': '0', '92': '1', 

100%|██████████| 15/15 [00:00<00:00, 16.48it/s]

KeyError for binary_previous
{'155': '1', '95': '1', '179': '1', '181': '0', '142': '0', '74': '0', '82': '0', '137': '1', '19': '1', '42': '0', '90': '0', '71': '1', '123': '0', '132': '0', '98': '0', '111': '1', '166': '1', '163': '1', '151': '0', '12': '1'}
{'155': '1', '95': '1', '179': '1', '181': '0', '142': '0', '74': '0', '82': '0', '137': '1', '19': '1', '42': '0', '90': '0', '71': '1', '123': '0', '132': '0', '98': '0', '111': '1', '166': '1', '163': '1', '151': '0', '12': '1', '29': '0'}
['155', '95', '179', '181', '142', '74', '82', '137', '19', '42', '90', '71', '123', '132', '98', '111', '166', '163', '151', '12', '29']
KeyError for binary_previous
{'115': '0', '119': '1', '103': '0', '137': '1', '20': '1', '182': '1', '66': '0', '78': '0', '16': '1', '157': '1', '166': '1', '164': '0', '152': '1', '46': '1', '124': '1', '133': '0', '80': '1', '71': '1', '154': '0', '52': '0', '99': '0', '40': '1', '138': '1', '171': '0', '49': '1', '48': '0', '108': '0', '3': '0', '29': 

In [10]:
results_list[-1].get('cascade history')

[{'Alice blocks': [{'37': '1',
    '71': '1',
    '139': '1',
    '188': '1',
    '155': '0',
    '88': '1',
    '142': '0',
    '136': '0',
    '182': '1',
    '115': '1',
    '47': '1',
    '73': '1',
    '83': '0',
    '39': '0',
    '55': '1',
    '80': '0',
    '43': '0',
    '35': '1',
    '95': '0',
    '22': '0',
    '14': '1',
    '168': '0',
    '172': '0',
    '58': '0',
    '98': '1',
    '74': '1',
    '113': '1',
    '18': '0'},
   {'143': '1',
    '45': '1',
    '51': '1',
    '131': '0',
    '125': '1',
    '117': '0',
    '122': '0',
    '190': '0',
    '56': '0',
    '64': '1',
    '1': '0',
    '79': '1',
    '72': '1',
    '132': '0',
    '86': '0',
    '48': '0',
    '28': '0',
    '184': '1',
    '0': '0',
    '91': '0',
    '118': '0',
    '41': '1',
    '141': '0',
    '29': '1',
    '38': '1',
    '11': '1',
    '110': '0',
    '183': '1'},
   {'30': '1',
    '12': '1',
    '102': '0',
    '52': '0',
    '189': '0',
    '145': '1',
    '111': '1',
    '180': '1